Primeiro, vamos realizar a importação de todas as bibliotecas que usaremos.

In [135]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, make_scorer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from scipy.stats import uniform, randint, loguniform
import xgboost as xgb
import lightgbm as lgb
import joblib

Agora, vamos importar nossos dados.

In [2]:
df_test = pd.read_csv('../test.csv')
df = pd.read_csv('../train.csv')

In [3]:
pd.set_option('display.max_columns', None) # Configuracao para mostrar todas as colunas

## Entendendo o significado de todas as colunas

É de extrema importância entender todas as colunas, por isso, vamos copiar e colar o significado delas:

- track_id: O ID único de cada música
    
- artists: Nome dos(as) artistas que performaram a música, separados por ';'
    
- album_name: Nome do álbum no qual aparece a música
    
- track_name: Nome da música
    
- duration_ms: A duração da música em milissegundos
    
- explicit: Boolean indicando se a música possui conteúdo explícito
    
- danceability: Descreve quanto uma música é "dançante" (0.0 = menos dançante, 1.0 = mais dançante)
    
- energy: Representa a intensidade e atividade de uma música (0.0 = baixa energia, 1.0 = alta energia)
    
- key: A tonalidade musical da faixa mapeada usando a notação padrão de Classe de Altura (12 notas musicais)
    
- loudness: Nível geral de volume da faixa em decibéis (dB)
    
- mode: Indica a modalidade (maior ou menor) da faixa
    
- speechiness: Detecta a presença de palavras faladas na faixa
    
- acousticness: Medida de confiança sobre se a faixa é acústica (0,0 = não acústica, 1,0 = altamente acústica)
    
- instrumentalness: Prediz se uma faixa contém vocais (0,0 = contém vocais, 1,0 = instrumental)
    
- liveness: Detecta a presença de uma audiência na gravação (0,0 = gravação em estúdio, 1,0 = performance ao vivo)
    
- valence: Mede a positividade musical transmitida por uma faixa (0,0 = negativa, 1,0 = positiva)
    
- tempo: Tempo estimado da faixa em batidas por minuto (BPM)
    
- time_signature: Assinatura de tempo estimada da faixa (de 3 a 7)
    
- track_genre: O gênero da música
    
- popularity_target: Boolean indicando se a música é popular ou não

Realizando a análise das colunas, fiquei em dúvida sobre a diferença entre a coluna 'tempo' e 'time_signature'. Por isso, realizei uma análise mais profunda no significado delas:

* tempo (Batidas por minuto):
   * O tempo representa a velocidade ou ritmo de uma faixa, medido em batidas por minuto (BPM).
   * O tempo médio é de cerca de 122 BPM, o que representa um ritmo moderado e enérgico.
   * O tempo mínimo é 0 BPM (o que pode indicar dados ausentes ou faixas muito lentas), e o máximo é cerca de 223 BPM (muito rápido).

* time_signature (Fórmula de compasso):
   * A fórmula de compasso representa a estrutura rítmica de uma faixa, indicando quantas batidas existem em cada compasso e qual valor de nota representa uma batida.
   * Os valores geralmente variam de 3 a 7, sendo 4 o mais comum (representando o compasso 4/4, que é padrão em muitos gêneros).
   * A partir das estatísticas, podemos ver que a mediana e os percentis 25 e 75 são 4, confirmando que o compasso 4/4 é realmente o mais comum.
   * O mínimo de 0 pode indicar dados ausentes ou fórmulas de compasso não convencionais.

Primeiro dou uma olhada por cima dos dados apenas para entender quais colunas são númericas e categóricas. Também tenho a intenção de ver elas como dados reais e por isso chamo o comando `df.head(3)`

In [ ]:
df.head(3)

In [ ]:
df.info()

Agora partindo para uma análise mais profunda das colunas numéricas, começo com uma análise da estatística descritiva de cada coluna.

In [ ]:
df.describe()

Com o output acima, já é possível observar que valores iguais a 0 em 'tempo' e 'time_signature' provavelmente são missing values. Chegamos a essa conclusão porque músicas com 0 BPM não existem, mesmo muito lentas. E 'time_sigture' já comenta na definição das colunas que os valores vão de 3 a 7, ou seja, 0 é incorreto. Não realizamos a correção desses valores faltantes nesse exato momento, mas em breve voltaremos com eles.

## Exploração e Visualização dos Dados

Agora, na intenção de entender os dados e descobrir padrões, vamos realizar uma exploração e visualização dos dados! Para isso, vamos utilizar bibliotecas como Matplot e Seaborn. Nossa missão aqui é descobrir padrões, correlações e tendências nos dados. Vamos usar visualizações eficazes para comunicar os insights e justificar nossas futuras escolhas de features e modelos.

In [7]:
# Definindo um estilo para nossos graficos e alterando o tamanho padrão das figuras geradas pelo Matplotlib
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

Para o começo da nossa análise, é importante saber a quantidade de músicas que são populares ou não e saber como está a distribuição entre elas. No gráfico gerado a seguir, é possível observar que existe uma divisão bem harmoniosa entre músicas populares e não populares.

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='popularity_target', data=df)
plt.title('Distribution of Song Popularity')
plt.xlabel('Popularity (0: Not Popular, 1: Popular)')
plt.ylabel('Count')
plt.show()

Partindo com nossa análise, vamos agora observar nossas variáveis numéricas e a relação entre elas. O Heatmap é um gráfico perfeito para isso que demonstra as relações entre as colunas. Resumindo bastante funciona da forma a seguir: 

Os valores de correlação variam de -1 a 1:

- **+1**: Correlação positiva perfeita. À medida que uma variável aumenta, a outra também aumenta.
- **-1**: Correlação negativa perfeita. À medida que uma variável aumenta, a outra diminui.
- **0**: Nenhuma correlação. As duas variáveis não afetam uma à outra.

In [ ]:
numerical_features = ['duration_ms', 'danceability', 'energy', 'key', 'loudness', 'speechiness', 
                      'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
plt.figure(figsize=(12, 10))
sns.heatmap(df[numerical_features].corr(), annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap of Numerical Features')
plt.show()

Realizando a análise do Heatmap acima, é possível identificar alguns padrões importantes sobre nossa base de dados.

1. Correlação forte entre 'energy' e 'loudness': Existe uma correlação positiva forte (0.76) entre 'energy' e 'loudness', indicando que músicas mais enérgicas tendem a ser mais altas.

2. Correlação negativa entre 'acousticness' e 'energy': Há uma forte correlação negativa (-0.73) entre 'acousticness' e 'energy', sugerindo que músicas mais acústicas tendem a ser menos enérgicas.

3. Correlação negativa entre 'acousticness' e 'loudness': Similarmente, existe uma correlação negativa moderada (-0.59) entre 'acousticness' e 'loudness', indicando que músicas acústicas tendem a ser menos altas.

4. Correlação positiva entre 'danceability' e 'valence': Há uma correlação positiva moderada (0.48) entre 'danceability' e 'valence', sugerindo que músicas mais dançantes tendem a ter um tom emocional mais positivo.

5. Correlação negativa entre 'instrumentalness' e 'loudness': Observa-se uma correlação negativa moderada (-0.43) entre 'instrumentalness' e 'loudness', indicando que músicas mais instrumentais tendem a ser menos altas.

6. Pouca correlação com 'duration_ms': A 'duration_ms' tem correlações fracas com a maioria das outras características, sugerindo que o comprimento da música não está fortemente relacionado com suas outras propriedades acústicas.

7. Correlações fracas com 'key': A 'key' tem correlações muito fracas com outras características, indicando que não há uma relação forte entre a tonalidade e outros aspectos musicais neste conjunto de dados.

8. Correlações moderadas com 'valence': A 'valence' tem correlações moderadas positivas com 'danceability' (0.48) e 'energy' (0.26), sugerindo que músicas mais positivas tendem a ser mais dançantes e enérgicas.

9. Pouca correlação entre 'tempo' e outras características: O 'tempo' da música tem correlações relativamente fracas com outras características, com a mais forte sendo com 'energy' (0.24).

10. Correlação fraca entre 'speechiness' e outras características: 'Speechiness' tem correlações geralmente fracas com outras características, com a mais notável sendo com 'liveness' (0.21).

11. 'Liveness' tem correlações fracas: A característica 'liveness' não mostra correlações fortes com nenhuma outra característica, sugerindo que é relativamente independente das outras propriedades musicais.

Saber desses padrão será importante para quando começarmos a selecionar as features para o treinamento do nosso modelo! Porque aqui é possível observar que alguns colunas estão extremamente ligadas com outras.

Agora, queremos entender como diferentes características musicais se relacionam com a popularidade das músicas. Por isso, vamos criar alguns gráficos para realizar essa visualização.

Com essa análise, vamos identificar quais características musicais têm maior influência na popularidade de uma música.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness']
for i, feature in enumerate(features):
    sns.boxplot(x='popularity_target', y=feature, data=df, ax=axes[i//3][i%3])
    axes[i//3][i%3].set_title(f'{feature.capitalize()} by Popularity')
plt.tight_layout()
plt.show()

Observando o gráfico acima, conseguimos ter alguns insights.

1. Danceability (Dançabilidade):

Há uma ligeira tendência de músicas populares (1) terem maior dançabilidade.
A diferença é pequena, mas notável, o que sugere que músicas mais dançantes têm uma probabilidade um pouco maior de serem populares.

2.Instrumentalness (Instrumentalidade):

Músicas populares tendem a ter menor instrumentalidade.
Isso sugere que músicas com vocais são geralmente mais populares do que músicas puramente instrumentais.

Agora, para entender quais gêneros proporcionam mais sucessos às músicas, vamos criar um gráfico para visualizar os 10 melhores gêneros em termos de músicas populares.

In [ ]:
top_genres = df['track_genre'].value_counts().nlargest(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_genres.index, y=top_genres.values)
plt.title('Top 10 Genres by Count')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.scatterplot(data=df, x='energy', y='danceability', hue='popularity_target', palette='viridis')
plt.title('Energy vs. Danceability (Colored by Popularity)')
plt.show()

Análisando o gráfico acima, podemos observar que a popularidade não parece ser exclusivamente determinada por altos níveis de energia e dançabilidade, já que músicas populares existem em diversos níveis de energia e dançabilidade, sugerindo que outros fatores também influenciam a popularidade.

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(data=df, x='duration_ms', bins=50, kde=True)
plt.title('Distribution of Song Durations')
plt.xlabel('Duration (ms)')
plt.show()

Observando a duração das músicas, podemos observar que há uma concentração muito alta de músicas com duração menor. Também observamos que existem músicas com durações extremamente longas. Estes podem ser álbuns inteiros, performances ao vivo, ou erros nos dados.

- A assimetria da distribuição pode afetar análises estatísticas, sendo necessário considerar transformações ou métodos robustos. Em breve voltaremos nesse tópico ao tratar os dados.

Já o gráfico de barras empilhadas abaixo apresenta uma comparação visual da proporção de conteúdo explícito entre músicas populares e não populares 

In [ ]:
explicit_by_popularity = df.groupby('popularity_target')['explicit'].value_counts(normalize=True).unstack()
explicit_by_popularity.plot(kind='bar', stacked=True)
plt.title('Proportion of Explicit Content by Popularity')
plt.xlabel('Popularity')
plt.ylabel('Proportion')
plt.legend(title='Explicit', loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()
plt.show()

Observando o gráfico acima, podemos observar que:

1. A maioria das músicas, tanto populares quanto não populares, não contém conteúdo explícito.
2. Há uma ligeira tendência de músicas populares terem uma proporção um pouco maior de conteúdo explícito em comparação com as não populares.
3. A presença de conteúdo explícito não parece ser um fator determinante para a popularidade de uma música, dada a pequena diferença observada.

Agora para fechar nossa análise, vamos comparar a distribuição da duração das músicas entre as categorias de popularidade.

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxenplot(x='popularity_target', y='duration_ms', data=df)
plt.title('Song Duration Distribution by Popularity')
plt.xlabel('Popularity (0: Not Popular, 1: Popular)')
plt.ylabel('Duration (ms)')
plt.show()

Observando o gráfico acima, vemos que:

1. As medianas de duração para músicas populares e não populares são muito semelhantes, sugerindo que a duração por si só não é um forte indicador de popularidade.
2. A dispersão (representada pelo tamanho das caixas) é ligeiramente menor para músicas populares, indicando uma maior consistência na duração dessas faixas.
3. Existem muitos outliers em ambas as categorias, representando músicas com durações excepcionalmente longas.
4. As músicas populares parecem ter menos outliers extremos em comparação com as não populares, especialmente na faixa de duração mais longa.
5. A maioria das músicas, independentemente da popularidade, tem duração dentro de um intervalo relativamente estreito, como evidenciado pelo tamanho das caixas do boxplot.
6. Há uma leve tendência de músicas populares terem durações um pouco mais curtas, observável pela posição ligeiramente inferior da caixa para músicas populares.

Os insights mais importantes que encontramos em nossa análise incluem: a ligeira tendência de músicas populares terem maior energia e dançabilidade; a predominância de conteúdo não explícito em ambas as categorias de popularidade, com uma sutil inclinação para mais conteúdo explícito em músicas populares; e a observação de que a duração das músicas não é um forte indicador de popularidade, embora músicas populares tendam a ter durações mais consistentes. Além disso, notamos correlações significativas entre certas características musicais, como a relação positiva entre energia e volume (loudness), e negativa entre acústica e energia.

Ainda é muito cedo para entrarmos na escolha das features para nosso modelo, mas já é possível observar que não existe uma coluna em específica que determina a popularidade das músicas, com isso, é bastante provável que vamos utilizar todas as colunas para treinar nosso modelo, excluindo apenas as colunas de identificação.

## Formulação de Hipóteses

#### Hipótese 01: Influência da dançabilidade na era do TikTok

Com a popularização do TikTok, onde coreografias virais são frequentes, músicas mais dançantes podem ter maior probabilidade de se tornarem populares. Esta hipótese sugere uma correlação positiva entre a dançabilidade de uma música e sua popularidade.

In [ ]:
plt.subplot(2, 2, 1)
sns.boxplot(x='popularity_target', y='danceability', data=df)
plt.title('Dançabilidade vs. Popularidade')
plt.xlabel('Popularidade (0: Não Popular, 1: Popular)')
plt.ylabel('Dançabilidade')

É possível observar que a média de músicas dançantes é um pouco mais alto para músicas populares, mas a diferença não é tão grande assim, logo, não indica extrema importância se a música é dançante ou não para ela ser popular.

#### Hipótese 02: Conteúdo explícito em músicas de temática triste

Músicas categorizadas como "sad" ou emocionalmente intensas podem ter uma maior tendência a conter conteúdo explícito.

In [ ]:
plt.subplot(2, 2, 2)
sad_genres = ['sad', 'melancholic', 'emotional']  # Ajuste conforme necessário
df['is_sad'] = df['track_genre'].isin(sad_genres)
sns.barplot(x='is_sad', y='explicit', data=df)
plt.title('Conteúdo Explícito em Músicas Tristes vs. Outras')
plt.xlabel('Gênero Triste')
plt.ylabel('Proporção de Conteúdo Explícito')
df = df.drop(columns='is_sad')

É possível observar que o conteúdo explícito está extremamente presente em músicas classificadas como 'sad', 'melancholic' ou 'emotional'. Comprovando nossa hipótese.

#### Hipótese 03: Popularidade de músicas com temática melancólica

Músicas classificadas como 'sad' estão entre as mais populares, principalmente porque hoje a depressão é o problema do século.

In [ ]:
top_genres = df['track_genre'].value_counts().nlargest(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_genres.index, y=top_genres.values)
plt.title('Top 10 Genres by Count')
plt.xticks(rotation=45, ha='right')
plt.show()

É possível ver que o gênero 'sad' é o segundo gênero com maior sucesso de músicas populares. Comprovando nossa hipótese.

#### Hipótese 04: Relação inversa entre instrumentalidade e popularidade

Músicas com alto grau de instrumentalidade (pouco ou nenhum vocal) podem tender a ser menos populares, possivelmente devido à preferência do público mainstream por músicas com letras e vozes.

In [ ]:
plt.subplot(2, 2, 4)
sns.boxplot(x='popularity_target', y='instrumentalness', data=df)
plt.title('Instrumentalidade vs. Popularidade')
plt.xlabel('Popularidade (0: Não Popular, 1: Popular)')
plt.ylabel('Instrumentalidade')

É possível observar que existem músicas populares e que são instrumentais, mas, em maioria, elas são classificadas como não populares. Comprovando nossa hipótese.

## Limpeza e Tratamento de Valores Nulos

### Tratamento de valores duplicados

Fazer a limpeza de valores duplicados é importantes pois valores duplicados podem prejudicar no aprendizado do nosso algoritmo. Para realizar a limpeza, vamos primeiro checar se existem valores duplicados e caso existirem, vamos excluir todos.

In [ ]:
duplicated_values = df[df.duplicated()]
print(len(duplicated_values))

Como podemos ver acima, não existem valores duplicados. Assim, seguiremos com nossa análise

### Tratamento de missing values

Missing values são valores = null

É importante verificar a existência deles e resolver os valores faltantes caso existam.

In [ ]:
df.isnull().sum()

Como podemos ver acima, também não existem valores faltantes.

Agora, vamos verificar valores que são iguais a 0. Como identificamos lá no começo desse notebook, já sabemos de alguns valores que não deveriam estar iguais a 0. Também vamos procurar por novos valores iguais a 0 e que não deveriam estar assim, caso existam, vamos resolver.

In [ ]:
zero_count = (df == 0).sum()
print(zero_count)

Como podemos ver acima, as colunas 'time_signature' e 'tempo' possuem valores iguais a 0 e elas não deveriam estar assim:

1. Tempo:

- Representa a velocidade da música em batidas por minuto (BPM).
- Um valor 0 significaria que não há batidas, o que é musicalmente impossível.
- Tempos típicos variam de cerca de 60 BPM (lento) a 200 BPM (muito rápido).


2. Time signature (fórmula de compasso):

- Indica quantas batidas há por compasso e qual nota representa uma batida.
- É escrita como uma fração, por exemplo, 4/4 ou 3/4.
- Um valor 0 não faria sentido, pois implicaria em nenhuma batida por compasso.

In [ ]:
df['time_signature'].value_counts()

In [ ]:
print(df['tempo'].describe(), df['tempo'].value_counts())

Para resolver esses 2 problemas, vamos: 

- Substituir os valores de 'time_signature' pela moda, que é 4
- Substituir os valores de 'tempo' pela mediana

In [25]:
tempo_median = df['tempo'].median()
df_treating_data = df.copy()
df_treating_data['tempo'] = df_treating_data['tempo'].replace(0,tempo_median)
df_treating_data['time_signature'] = df_treating_data['time_signature'].replace(0,4)

### Identificação de outliers e correção

As colunas que vamos visualizar e depois tratar os outliers são as colunas: 'duration_ms', 'loudness' e 'tempo'.

Realizei a escolha dessas três colunas porque todas as outras colunas numéricas se constituem de valores que vâo de 0 a 1. Algumas até possuem outliers mas são features importantes e que quero manter para preservar a pureza dos dados em features que vão de 0 a 1. Mais tarde, vamos minimizar os impactos que esses outliers podem causar com o standardScaler

Agora partindo para a identificação dos outliers que escolhemos, vamos começar visualizando em boxplot os possíveis outliers.

In [ ]:
sns.boxplot(data=df_treating_data, y='duration_ms')
plt.show()

Observando o gráfico acima, é possívei ver que 'duration_ms' possui vários outliers, vamos guardar essa informação e em breve corrigir eles.

Agora fazendo a análise dos outliers de 'loudness', vamos desenhar outro boxplot para realizar essa visualização:

In [ ]:
sns.boxplot(data=df_treating_data, y='loudness')
plt.show()

Agora fazendo a análise dos outliers de 'tempo', vamos desenhar outro boxplot para realizar essa visualização:

In [ ]:
sns.boxplot(data=df_treating_data, y='tempo')
plt.show()

Sabendo dos outliers que 'duration_ms', 'loudness' e 'tempo' possuem, vamos agora partir para a correção deles.

Para os outliers de 'loudness' e 'tempo', decidi manter eles do jeito que estão. Pelas mesmas que citei acima, sobre as colunas que vão de 0 a 1. 

Como essas duas colunas representam caracteristicas específicas da música em questão e também podem representar escolhas específicas do artista que a fez, acho importante manter esses outliers do jeito que estão.

---

Já para os outliers de 'duration_ms', vamos agora tentar três medidas diferentes para corrigi-los:

1. Vamos manter eles intocáveis.
2. Vamos excluir todos os outliers
3. Vamos aplicar uma transformação logarítmica.

A transformação logarítmica ajuda a lidar com outliers (valores extremos) e dados distorcidos ao comprimir a amplitude dos valores, especialmente os números grandes. Isso faz com que valores extremos se tornem menos extremos, enquanto ainda preserva as diferenças relativas entre os dados.

Antes de tomar essas três medidas, vamos excluir algumas colunas que não serão importantes para o treinamento do nosso modelo, essas colunas são: 'track_id', 'track_unique_id', 'artists', 'album_name' e 'track_name'.

- track_id: Usado apenas para identificação de uma música em questão, por isso, não é importante para nosso futuro modelo.
- track_unique_id: Também usado apenas para identificação.
- artists: Poderia ser um dado importante caso nossa base de dados classificasem o nível de fama de cada artista, pois artistas famosos tendem a gerar músicas populares com mais facilidade. Como nossa base de dados não contém isso, vamos excluir essa coluna porque na situação que está, essa coluna é usada apenas para identificação.
- album_name: Como nossa base de dados não contém também uma coluna classificando o nível de popularidade do album em questão, não vamos estar utilizando essa informação.
- track_name: Usado apenas para identificação.

In [29]:
df_treating_data = df_treating_data.drop(columns=['track_unique_id', 'track_id', 'artists', 'album_name', 'track_name'])

Com essas alterações feitas, agora vamos tomar as três medidas para realizar a correção dos outliers de 'durations_ms'

1. Primeiro, vamos tomar a medida em que excluímos todos os outliers de 'duration_ms'

In [30]:
Q1 = df_treating_data['duration_ms'].quantile(0.25)
Q3 = df_treating_data['duration_ms'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_no_duration_outliers = df_treating_data.copy()

df_no_duration_outliers = df_no_duration_outliers[(df_no_duration_outliers['duration_ms'] >= lower_bound) & (df_no_duration_outliers['duration_ms'] <= upper_bound)]

2. Segundo, vamos aplicar a transformação logarítmica

In [31]:
# Criando uma copia para nossa segunda medida com os outliers
df_duration_log = df_treating_data.copy()

# fazendo a transformacao logaritma
df_duration_log['duration_log'] = np.log1p(df_duration_log['duration_ms'])

# excluindo a coluna de duracao normal porque nao vamos utilizar ela nessa base de dados
df_duration_log = df_duration_log.drop(columns=['duration_ms'])

Agora é possível observar que estamos com três base de dados:

1. df_treating_data: Contém os outliers de 'duration_ms' sem alterações
2. df_no_duration_outliers: Não contém nenhum outlier de 'duration_ms'
3. df_duration_log: Contém os outliers mas com a transformação log aplicada em todos.

Agora, seguiremos com esses 3 modelos até o treinamento do nosso modelo, onde veremos qual base de dados resultará um melhor resultado.

## Codificação de Variáveis Categóricas

Aplique técnicas apropriadas de codificação para transformar variáveis categóricas em formatos utilizáveis em modelos preditivos, garantindo que a informação essencial não seja perdida no processo.

Agora, entramos na etapa de codificar as variáveis categóricas. As únicas 2 colunas que não são numéricas e precisar sem codificadas, são 'explicit' que é uma coluna boolean e 'track_genre' que é uma coluna categórica. 

Para garantir que não vamos perder as informações essenciais nessa etapa, vamos codificar as variáveis de 'track_genre' com um método chamado Target Encode. 

Já a coluna 'explicit', por ser uma coluna boolean, para codifica-lá, basta transformar os valores em valores int. Faremos isso abaixo.

É importante lembrar que, como temos 3 base de dados, vamos codificar as colunas de cada uma:

In [32]:
df_treating_data['explicit'] = df_treating_data['explicit'].astype(int)
df_no_duration_outliers['explicit'] = df_no_duration_outliers['explicit'].astype(int)
df_duration_log['explicit'] = df_duration_log['explicit'].astype(int)

Agora abaixo, criamos a função para codificar nossa coluna 'track_genre'. 

A escolha do Target Encode foi feita porque enfrentamos o desafio de lidar com a variável categórica 'track_genre', que possui alta cardinalidade (114 categorias únicas). Para abordar esse problema de maneira eficaz, implementamos a técnica de Target Encoding.

Target Encoding é uma técnica usada para converter variáveis categóricas em valores numéricos. Diferentemente de métodos como mais utilizados como LabelEncoder, o Target Encoding leva em consideração a variável alvo (popularity_target) ao realizar a codificação.

1. Para cada categoria na variável categórica (no nosso caso, cada gênero musical), calculamos a média da variável alvo.
2. Substituímos cada categoria pelo valor médio calculado.
3. Para evitar overfitting, utilizamos uma abordagem de validação cruzada na implementação.

Mas por que estamos usando o Target Encoder nesse caso ?

1. Alta Cardinalidade: Com 114 gêneros musicais únicos, métodos como One-Hot Encoding criariam um número excessivo de features, potencialmente levando a problemas de dimensionalidade.
2. Captura de Informação Relevante: O Target Encoding captura a relação entre cada gênero musical e a popularidade da música, fornecendo uma representação numérica significativa.
3. Eficiência Computacional: Reduz a dimensionalidade dos dados sem perder informações cruciais sobre a relação entre gênero e popularidade.
4. Tratamento de Categorias Raras: Lida bem com gêneros musicais que aparecem com pouca frequência no dataset.

O Target Encoding nos permite transformar a informação categórica dos gêneros musicais em uma representação numérica que preserva e destaca a relação entre gênero e popularidade.


In [33]:
def target_encode_train(df, column, target, n_splits=5):
    encoded = np.zeros(len(df))
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    encoding_dict = {}
    
    for train_idx, val_idx in kf.split(df):
        target_means = df.iloc[train_idx].groupby(column)[target].mean()
        encoded[val_idx] = df[column].iloc[val_idx].map(target_means)
        
        encoding_dict.update(target_means.to_dict())
    
    global_mean = df[target].mean()
    encoded = np.where(np.isnan(encoded), global_mean, encoded)
    
    encoding_dict['_global_mean'] = global_mean
    
    return encoded, encoding_dict

In [34]:
def target_encode_test(df, column, encoding_dict):
    global_mean = encoding_dict['_global_mean']
    return df[column].map(encoding_dict).fillna(global_mean)

In [ ]:
df_treating_data['track_genre_encoded'], encoding_dict = target_encode_train(df_treating_data, 'track_genre', 'popularity_target')

# Verificação dos resultados
print(df_treating_data[['track_genre', 'track_genre_encoded', 'popularity_target']].head(10))
print("\nCorrelação entre 'track_genre_encoded' e 'popularity_target':")
print(df_treating_data['track_genre_encoded'].corr(df['popularity_target']))
df_treating_data = df_treating_data.drop(columns='track_genre')

# Aqui vamos salvar o encoding_dict para usarmos mais tarde quando for para codificar o df_test
joblib.dump(encoding_dict, 'category_encoding.joblib')

In [ ]:
df_no_duration_outliers['track_genre_encoded'], encoding_dict_no_duration_outliers = target_encode_train(df_no_duration_outliers, 'track_genre', 'popularity_target')

# Verificação dos resultados
print(df_no_duration_outliers[['track_genre', 'track_genre_encoded', 'popularity_target']].head(10))
print("\nCorrelação entre 'track_genre_encoded' e 'popularity_target':")
print(df_no_duration_outliers['track_genre_encoded'].corr(df['popularity_target']))
df_no_duration_outliers = df_no_duration_outliers.drop(columns='track_genre')

In [ ]:
df_duration_log['track_genre_encoded'], encoding_dict_log = target_encode_train(df_duration_log, 'track_genre', 'popularity_target')

# Verificação dos resultados
print(df_duration_log[['track_genre', 'track_genre_encoded', 'popularity_target']].head(10))
print("\nCorrelação entre 'track_genre_encoded' e 'popularity_target':")
print(df_duration_log['track_genre_encoded'].corr(df['popularity_target']))
df_duration_log = df_duration_log.drop(columns='track_genre')

Análisando os resultados acima, podemos ver que o Target Encoding capturou informações relevantes sobre a relação entre os gêneros musicais e a popularidade, sem criar uma relação perfeita que poderia levar a overfitting.

Por exemplo, "pop" tem um valor codificado de 0.675627, sugerindo que cerca de 67.56% das músicas pop são populares.

Essas informações serão valiosas no momento em que formos treinar os algoritmos.

## Seleção de Features

As features que vamos incluir em nosso modelo preditivo são: 

duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'popularity_target', 'track_genre_encoded'

Justificativa para a escolha dessas features:

1. duration_ms: Duração da faixa em milissegundos
2. explicit: Indicador de conteúdo explícito
3. danceability: Quão adequada a faixa é para dançar
4. energy: Medida de intensidade e atividade
5. key: Tonalidade da faixa
6. loudness: Volume geral da faixa em decibéis
7. mode: Modalidade da faixa (maior ou menor)
8. speechiness: Presença de palavras faladas
9. acousticness: Medida de quão acústica é a faixa
10. instrumentalness: Predição da ausência de vocais
11. liveness: Presença de público na gravação
12. valence: Positividade musical da faixa
13. tempo: Velocidade ou ritmo estimado em BPM
14. time_signature: Fórmula de compasso estimada
15. popularity_target: Alvo de popularidade (variável dependente)
16. track_genre_encoded: Gênero da faixa (codificado)

Justificativa da Seleção

1. Características Acústicas: danceability, energy, loudness, acousticness, instrumentalness, valence, e tempo foram incluídas por representarem aspectos fundamentais do som e da percepção musical, que podem influenciar diretamente a popularidade.
2. Estrutura Musical: key, mode, e time_signature foram mantidas por fornecerem informações sobre a estrutura musical, que pode afetar a acessibilidade e apelo da faixa.
3. Conteúdo: explicit e speechiness foram incluídas por poderem influenciar a recepção e distribuição da música em diferentes plataformas e faixas etárias.
4. Experiência de Escuta: duration_ms e liveness foram mantidas por afetarem a experiência de escuta, potencialmente impactando o engajamento do ouvinte.
5. Contexto: track_genre_encoded foi incluída por fornecer contexto importante sobre o estilo musical, que pode ser crucial para entender padrões de popularidade em diferentes nichos.
6. Alvo: popularity_target é a variável dependente que o modelo tentará prever.

- Vale também levantar o ponto de que para prever com a melhor acurácia possível, é importante manter o máximo de colunas porque os dados parecem estar extremamente conectados entre si.

Pré-processamento e transformação:

1. Para tentar ao máximo garantir o melhor resultado possível, em breve, vamos criar novas base de dados e realizar o escalonamento de todos os valores. (Vale levantar o ponto de que vamos testar todas as bases de dados e escolher o melhor resultado gerado.)

## Construção e Avaliação do Modelo

Antes de selecionar um modelo de Machine Learning, precisamos separar nossos dados entre 'treino', 'validação' e 'teste'. Como estamos trabalhando com três bases de dados, vamos separar os dados para todas elas.

Os três códigos abaixo, separam o 'popularity_target' do resto da base de dados, assim, podemos treinar nosso algoritmo.

In [38]:
X_df_treating_data = df_treating_data.drop('popularity_target', axis=1)
y_df_treating_data = df_treating_data['popularity_target']

In [39]:
X_df_no_duration_outliers = df_no_duration_outliers.drop('popularity_target', axis=1)
y_df_no_duration_outliers = df_no_duration_outliers['popularity_target']

In [40]:
X_df_duration_log = df_duration_log.drop('popularity_target', axis=1)
y_df_duration_log = df_duration_log['popularity_target']

Agora, vamos separar as bases de dados entre 'train', 'validation' e 'test'

In [ ]:
# Primeiro, conjunto de teste
X_temp_df_treating_data, X_test_df_treating_data, y_temp_df_treating_data, y_test_df_treating_data = train_test_split(X_df_treating_data, y_df_treating_data, test_size=0.2, random_state=42)

# Agora, o restante em treino e validação
X_train_df_treating_data, X_val_df_treating_data, y_train_df_treating_data, y_val_df_treating_data = train_test_split(X_temp_df_treating_data, y_temp_df_treating_data, test_size=0.25, random_state=42)

print(f"Tamanho do conjunto de treino: {len(X_train_df_treating_data)}")
print(f"Tamanho do conjunto de validação: {len(X_val_df_treating_data)}")
print(f"Tamanho do conjunto de teste: {len(X_test_df_treating_data)}")

In [ ]:
# Primeiro, conjunto de teste
X_temp_df_no_duration_outliers, X_test_df_no_duration_outliers, y_temp_df_no_duration_outliers, y_test_df_no_duration_outliers = train_test_split(X_df_no_duration_outliers, y_df_no_duration_outliers, test_size=0.2, random_state=42)

# Agora, o restante em treino e validação
X_train_df_no_duration_outliers, X_val_df_no_duration_outliers, y_train_df_no_duration_outliers, y_val_df_no_duration_outliers = train_test_split(X_temp_df_no_duration_outliers, y_temp_df_no_duration_outliers, test_size=0.25, random_state=42)

print(f"Tamanho do conjunto de treino: {len(X_train_df_no_duration_outliers)}")
print(f"Tamanho do conjunto de validação: {len(X_val_df_no_duration_outliers)}")
print(f"Tamanho do conjunto de teste: {len(X_test_df_no_duration_outliers)}")

In [ ]:
# Primeiro, conjunto de teste
X_temp_df_duration_log, X_test_df_duration_log, y_temp_df_duration_log, y_test_df_duration_log = train_test_split(X_df_duration_log, y_df_duration_log, test_size=0.2, random_state=42)

# Agora, o restante em treino e validação
X_train_df_duration_log, X_val_df_duration_log, y_train_df_duration_log, y_val_df_duration_log = train_test_split(X_temp_df_duration_log, y_temp_df_duration_log, test_size=0.25, random_state=42)

print(f"Tamanho do conjunto de treino: {len(X_train_df_duration_log)}")
print(f"Tamanho do conjunto de validação: {len(X_val_df_duration_log)}")
print(f"Tamanho do conjunto de teste: {len(X_test_df_duration_log)}")

Como um último teste, em busca do melhor resultado possível, vamos também escalonar nossa base de dados 'df_treating_data' para também testar algoritmos com o escalonamento e decidir entre o melhor resultado.

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train_df_treating_data)

In [45]:
X_train_df_treating_data_scaled = scaler.transform(X_train_df_treating_data)
X_val_df_treating_data_scaled = scaler.transform(X_val_df_treating_data)
X_test_df_treating_data_scaled = scaler.transform(X_test_df_treating_data)

### Testando os modelos preditivos

Agora, com tudo feito e separado, podemos começar a testar os algoritmos.

Nesta etapa, vamos testar vários algoritmos e apenas no final tirar nossas conclusões sobre o quais são os melhores, por isso, voltarei a documentar com frequência após essa etapa de testes.

#### RandomForest

In [46]:
rf_classifier = RandomForestClassifier(random_state=42, n_estimators=500, min_samples_split=2, min_samples_leaf=1,max_features='sqrt',max_depth=30)

In [ ]:
rf_classifier.fit(X_train_df_treating_data, y_train_df_treating_data)

In [ ]:
y_pred_df_treating_data_random_forest = rf_classifier.predict(X_val_df_treating_data)

print(classification_report(y_val_df_treating_data, y_pred_df_treating_data_random_forest))

In [ ]:
rf_classifier.fit(X_train_df_no_duration_outliers, y_train_df_no_duration_outliers)

In [ ]:
y_pred_df_no_duration_outliers_random_forest = rf_classifier.predict(X_val_df_no_duration_outliers)

print(classification_report(y_val_df_no_duration_outliers, y_pred_df_no_duration_outliers_random_forest))

In [ ]:
rf_classifier.fit(X_train_df_duration_log, y_train_df_duration_log)

In [ ]:
y_pred_df_duration_log_random_forest = rf_classifier.predict(X_val_df_duration_log)

print(classification_report(y_val_df_duration_log, y_pred_df_duration_log_random_forest))

In [ ]:
rf_classifier.fit(X_train_df_treating_data_scaled, y_train_df_treating_data)

In [ ]:
y_pred_df_treating_data_scaled_random_forest = rf_classifier.predict(X_val_df_treating_data_scaled)

print(classification_report(y_val_df_treating_data, y_pred_df_treating_data_scaled_random_forest))

Como podemos ver acima, o random forest em média nos garante um resultado de 82% de 'accuracy'. O ideial agora, é rodar o .predict na base de testar para garantir que não existe um overfitting.

In [ ]:
y_pred_df_treating_data_scaled_test = rf_classifier.predict(X_test_df_treating_data_scaled)

print(classification_report(y_test_df_treating_data, y_pred_df_treating_data_scaled_test))

Como podemos ver acima, o resultado do 'test' é bem parecido com o resultado do 'val'. O resultado dos 82% de 'acurracy' já são o suficiente para realizar o envio da tarefa mas vamos testar mais alguns modelos na busca de um 'accuracy' melhor.

#### Regressão Logística

In [56]:
logistic_model = LogisticRegression(C=0.01, class_weight='balanced', max_iter=100, penalty='l1', solver='saga')

In [ ]:
logistic_model.fit(X_train_df_treating_data, y_train_df_treating_data)

y_pred_df_treating_data_logistic_reggresion = logistic_model.predict(X_val_df_treating_data)

print(classification_report(y_val_df_treating_data, y_pred_df_treating_data_logistic_reggresion))

In [ ]:
logistic_model.fit(X_train_df_no_duration_outliers, y_train_df_no_duration_outliers)

y_pred_df_no_duration_outliers_logistic_reggresion = logistic_model.predict(X_val_df_no_duration_outliers)

print(classification_report(y_val_df_no_duration_outliers, y_pred_df_no_duration_outliers_logistic_reggresion))

In [ ]:
logistic_model.fit(X_train_df_duration_log, y_train_df_duration_log)

y_pred_df_duration_log_logistic_reggresion = logistic_model.predict(X_val_df_duration_log)

print(classification_report(y_val_df_duration_log, y_pred_df_duration_log_logistic_reggresion))

In [ ]:
logistic_model.fit(X_train_df_treating_data_scaled, y_train_df_treating_data)

y_pred_df_treating_data_scaled_logistic_reggresion = logistic_model.predict(X_val_df_treating_data_scaled)

print(classification_report(y_val_df_treating_data, y_pred_df_treating_data_scaled_logistic_reggresion))

Para garantir que os resultados não são overfitting. Vamos testar no 'test' agora.

In [ ]:
logistic_model.fit(X_train_df_treating_data_scaled, y_train_df_treating_data)

y_pred_df_treating_data_scaled_logistic_reggresion_test = logistic_model.predict(X_test_df_treating_data_scaled)

print(classification_report(y_test_df_treating_data, y_pred_df_treating_data_scaled_logistic_reggresion_test))

Os resultados não são tão bons quanto RandomForest mas também são resultados bons.

#### Naive Bayes

In [62]:
nb_model = BernoulliNB(alpha=0.1, binarize=0.5)

In [ ]:
nb_model.fit(X_train_df_treating_data, y_train_df_treating_data)

y_pred_df_treating_data_naive_bayes = nb_model.predict(X_val_df_treating_data)


print(classification_report(y_val_df_treating_data, y_pred_df_treating_data_naive_bayes))

In [ ]:
nb_model.fit(X_test_df_no_duration_outliers, y_test_df_no_duration_outliers)

y_pred_df_no_duration_outliers_naive_bayes = nb_model.predict(X_val_df_no_duration_outliers)

print(classification_report(y_val_df_no_duration_outliers, y_pred_df_no_duration_outliers_naive_bayes))

In [ ]:
nb_model.fit(X_train_df_duration_log, y_train_df_duration_log)

y_pred_df_duration_log_naive_bayes = nb_model.predict(X_val_df_duration_log)

print(classification_report(y_val_df_duration_log, y_pred_df_duration_log_naive_bayes))

In [ ]:
nb_model.fit(X_train_df_treating_data_scaled, y_train_df_treating_data)

y_pred_df_treating_data_scaled_naive_bayes = nb_model.predict(X_val_df_treating_data_scaled)

print(classification_report(y_val_df_treating_data, y_pred_df_treating_data_scaled_naive_bayes))

Para garantir que os resultados não são overfitting. Vamos testar no 'test' agora.

In [ ]:
nb_model.fit(X_train_df_treating_data, y_train_df_treating_data)

y_pred_df_treating_data_naive_bayes_test = nb_model.predict(X_test_df_treating_data)

print(classification_report(y_test_df_treating_data, y_pred_df_treating_data_naive_bayes_test))

Os resultados não são tão bons quanto RandomForest mas também são resultados bons.

### Árvore de Decisão

In [68]:
dt_model = DecisionTreeClassifier(criterion='gini', max_depth=10, max_features=None, min_samples_leaf=10, min_samples_split=10, splitter='random')

In [ ]:
dt_model.fit(X_train_df_treating_data, y_train_df_treating_data)

y_pred_df_treating_data_decision_tree = dt_model.predict(X_val_df_treating_data)

print(classification_report(y_val_df_treating_data, y_pred_df_treating_data_decision_tree))

In [ ]:
dt_model.fit(X_train_df_no_duration_outliers, y_train_df_no_duration_outliers)

y_pred_df_no_duration_outliers_decision_tree = dt_model.predict(X_val_df_no_duration_outliers)

print(classification_report(y_val_df_no_duration_outliers, y_pred_df_no_duration_outliers_decision_tree))

In [ ]:
dt_model.fit(X_train_df_duration_log, y_train_df_duration_log)

y_pred_df_duration_log_decision_tree = dt_model.predict(X_val_df_duration_log)

print(classification_report(y_val_df_duration_log, y_pred_df_duration_log_decision_tree))

In [ ]:
dt_model.fit(X_train_df_treating_data_scaled, y_train_df_treating_data)

y_pred_df_treating_data_scaled_decision_tree = dt_model.predict(X_val_df_treating_data_scaled)

print(classification_report(y_val_df_treating_data, y_pred_df_treating_data_scaled_decision_tree))

Podemos ver que os resultados são bem iguais entre todos os testes. Vamos continuar com o resultado gerado pela base de dados 'df_treating_data_scaled' porque é o melhor entre todos. Para garantir que os resultados não são overfitting. Vamos testar no 'test' agora.

In [ ]:
dt_model.fit(X_train_df_treating_data_scaled, y_train_df_treating_data)

y_pred_df_treating_data_scaled_decision_tree_test = dt_model.predict(X_test_df_treating_data_scaled)

print(classification_report(y_test_df_treating_data, y_pred_df_treating_data_scaled_decision_tree_test))

Como podemos ver, os resultados são bons e não estão em overfitting. Agora vamos seguir com os próximos testes.

### K-Nearest Neighbors (KNN)

In [74]:
knn = KNeighborsClassifier(algorithm='auto', n_neighbors=11, p=1, weights='distance')

In [ ]:
knn.fit(X_train_df_treating_data_scaled, y_train_df_treating_data)

y_pred_df_treating_data_scaled_knn = knn.predict(X_val_df_treating_data_scaled)

print(classification_report(y_val_df_treating_data, y_pred_df_treating_data_scaled_knn))

In [ ]:
y_pred_df_treating_data_scaled_knn_test = knn.predict(X_test_df_treating_data_scaled)

print(classification_report(y_test_df_treating_data, y_pred_df_treating_data_scaled_knn_test))

Os resultados acima são excelentes e mostram não possuir overfitting. KNN se mostra um dos melhores algoritmos em nossos testes por enquanto.

Vamos continuar nossos testes.

### XGBoost

In [ ]:
xgb_model = xgb.XGBClassifier(eval_metric='mlogloss', tree_method='hist', device='cuda')

best_params = {
    'subsample': 1.0,
    'reg_lambda': 1,
    'reg_alpha': 0.1,
    'n_estimators': 200,
    'max_depth': 9,
    'learning_rate': 0.2,
    'gamma': 0.2,
    'colsample_bytree': 0.6
}

xgb_model.set_params(**best_params)

In [ ]:
xgb_model.fit(X_train_df_treating_data, y_train_df_treating_data)

y_pred_df_treating_data_xg_boost = xgb_model.predict(X_val_df_treating_data)

print(classification_report(y_val_df_treating_data, y_pred_df_treating_data_xg_boost))

In [ ]:
xgb_model.fit(X_train_df_no_duration_outliers, y_train_df_no_duration_outliers)

y_pred_df_no_duration_outliers_xg_boost = xgb_model.predict(X_val_df_no_duration_outliers)

print(classification_report(y_val_df_no_duration_outliers, y_pred_df_no_duration_outliers_xg_boost))

In [ ]:
xgb_model.fit(X_train_df_duration_log, y_train_df_duration_log)

y_pred_df_duration_log_xg_boost = xgb_model.predict(X_val_df_duration_log)

print(classification_report(y_val_df_duration_log, y_pred_df_duration_log_xg_boost))

In [ ]:
xgb_model.fit(X_train_df_treating_data_scaled, y_train_df_treating_data)

y_pred_df_treating_data_scaled_xg_boost = xgb_model.predict(X_val_df_treating_data_scaled)

print(classification_report(y_val_df_treating_data, y_pred_df_treating_data_scaled_xg_boost))

Como podemos ver, nesse teste, a base de dados 'df_treating_data' e 'df_treating_data_scaled' emparatam. Na tentativa de desempatar e garantir que não existe um overfitting. Vamos rodar o teste nas duas bases.

In [ ]:
xgb_model.fit(X_train_df_treating_data, y_train_df_treating_data)

y_pred_df_treating_data_xg_boost_test = xgb_model.predict(X_test_df_treating_data)

print(classification_report(y_test_df_treating_data, y_pred_df_treating_data_xg_boost_test))

In [ ]:
xgb_model.fit(X_train_df_treating_data_scaled, y_train_df_treating_data)

y_pred_df_treating_data_scaled_xg_boost_test = xgb_model.predict(X_test_df_treating_data_scaled)

print(classification_report(y_test_df_treating_data, y_pred_df_treating_data_scaled_xg_boost_test))

Como podemos ver, a accurary no 'df_treating_data' ganha por 1. Vamos fazer mais alguns testes para encontrar o melhor modelo.

### LightGBM

In [84]:
lgb_model = lgb.LGBMClassifier(colsample_bytree=0.8388271726494101, learning_rate=0.20226242344096437, max_depth=12, min_child_weight=0.07965014662270789, min_split_gain=0.041166121435752816, n_estimators=401, num_leaves=112, reg_alpha=0.48655517463505216, reg_lambda=0.3009676977704801, subsample=0.9238490659063205)

In [ ]:
lgb_model.fit(X_train_df_treating_data, y_train_df_treating_data)

y_pred_df_treating_data_light_gbm = lgb_model.predict(X_val_df_treating_data)

print(classification_report(y_val_df_treating_data, y_pred_df_treating_data_light_gbm))

In [ ]:
lgb_model.fit(X_train_df_no_duration_outliers, y_train_df_no_duration_outliers)

y_pred_df_no_duration_outliers_light_gbm = lgb_model.predict(X_val_df_no_duration_outliers)

print(classification_report(y_val_df_no_duration_outliers, y_pred_df_no_duration_outliers_light_gbm))

In [ ]:
lgb_model.fit(X_train_df_duration_log, y_train_df_duration_log)

y_pred_df_duration_log_light_gbm = lgb_model.predict(X_val_df_duration_log)

print(classification_report(y_val_df_duration_log, y_pred_df_duration_log_light_gbm))

In [ ]:
lgb_model.fit(X_train_df_treating_data_scaled, y_train_df_treating_data)

y_pred_df_treating_data_scaled_light_gbm = lgb_model.predict(X_val_df_treating_data_scaled)

print(classification_report(y_val_df_treating_data, y_pred_df_treating_data_scaled_light_gbm))

Aqui, podemos ver que o 'df_duration_log' apresenta aparentemente o melhor desempenho. Vamos rodar o teste para garantir que não existe overfitting.

In [ ]:
lgb_model.fit(X_train_df_duration_log, y_train_df_duration_log)

y_pred_df_duration_log_light_gbm_test = lgb_model.predict(X_test_df_duration_log)

print(classification_report(y_test_df_duration_log, y_pred_df_duration_log_light_gbm_test))

Como podemos ver, os resultados também são muito bons.

Com base nos resultados apresentados, podemos identificar os três melhores algoritmos:

1. Random Forest (melhor desempenho geral)
2. LightGBM
3. XGBoost

Agora, com base nesses resultados, vamos realizar o Finetuning de Hiperparâmetros para o Random Forest.

## Finetuning de Hiperparâmetros

O finetuning de hiperparâmetros é um processo crucial na otimização de modelos de machine learning. Consiste em ajustar os parâmetros que não são aprendidos diretamente dos dados, mas que influenciam significativamente o desempenho do modelo.

Importância:

1. Melhora o desempenho do modelo
2. Reduz o overfitting ou underfitting
3. Adapta o modelo às características específicas do problema

Métodos de Busca:

- RandomizedSearchCV:

1. Realiza uma busca aleatória no espaço de parâmetros
2. Testa um número predefinido de combinações aleatórias
3. Vantagens: Mais eficiente computacionalmente, especialmente para espaços de busca grandes
4. Desvantagens: Pode não encontrar a combinação ótima absoluta, mas frequentemente encontra uma solução muito boa

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'max_features': ['auto', 'sqrt', 'log2'],
}


n_iter_search = 100  
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=n_iter_search, cv=5, scoring=make_scorer(accuracy_score), n_jobs=-1, random_state=42, verbose=2)

random_search.fit(X_train_df_treating_data_scaled, y_train_df_treating_data)

print("Melhores parâmetros (RandomizedSearchCV):", random_search.best_params_)
print("Melhor score (RandomizedSearchCV):", random_search.best_score_)

Como podemos ver acima, os melhores parâmetros são esses: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None}

Agora, vamos ajustar nosso Random Forest para ter todos esses parâmetros.

Com os melhores parâmetros em mãos, vamos realizar o teste novamente e ver como são os resultados.

In [92]:
rf_classifier = RandomForestClassifier(random_state=42, n_estimators=500, min_samples_split=2, min_samples_leaf=1,max_features='log2',max_depth=None)

In [ ]:
rf_classifier.fit(X_train_df_treating_data_scaled, y_train_df_treating_data)

In [ ]:
y_df_treating_data_scaled_random_forest_2 = rf_classifier.predict(X_val_df_treating_data_scaled)

print(classification_report(y_val_df_treating_data, y_df_treating_data_scaled_random_forest_2))

In [ ]:
y_df_treating_data_scaled_random_forest_2_test = rf_classifier.predict(X_test_df_treating_data_scaled)

print(classification_report(y_test_df_treating_data, y_df_treating_data_scaled_random_forest_2_test))

Agora, com o nosso melhor modelo, vamos fazer o .predict para nosso df_test, salvar o resultado em CSV e enviar para a competição. 

In [96]:
df_test_treated = df_test.copy()

In [97]:
df_test_treated = df_test.drop(columns=['track_id', 'track_unique_id', 'artists', 'album_name', 'track_name'])
df_test_treated['explicit'] = df_test_treated['explicit'].astype(int)
encoding_dict = joblib.load('category_encoding.joblib')
df_test_treated['track_genre_encoded'] = target_encode_test(df_test, 'track_genre', encoding_dict)
df_test_treated = df_test_treated.drop(columns=['track_genre'])
df_test_treated_scaled = scaler.transform(df_test_treated)

In [98]:
final_predict_scaled = rf_classifier.predict(df_test_treated_scaled)
final_result = pd.DataFrame({ 'track_unique_id': df_test['track_unique_id'], 'popularity_target': final_predict_scaled })
final_result.to_csv('predictions_scaled.csv', index=False)

# Testando um Ensemble Model

Ensemble Model é uma técnica de aprendizado de máquina que combina múltiplos modelos base para melhorar o desempenho e a precisão das previsões em relação a um único modelo. A ideia por trás do ensemble é que, ao combinar diferentes modelos, os erros de um modelo podem ser compensados pelos acertos de outros, resultando em uma performance geral melhor.

Tipos Comuns de Ensemble Models:

1. Bagging (Bootstrap Aggregating)

O objetivo do bagging é reduzir a variância de um modelo. Para isso, são criados vários subconjuntos aleatórios de dados de treino (com substituição) e, para cada subconjunto, um modelo é treinado. Em seguida, as previsões dos modelos são combinadas (geralmente por votação ou média).

2. Boosting

No boosting, os modelos são treinados de forma sequencial, onde cada modelo tenta corrigir os erros do anterior. Assim, os modelos seguintes dão mais importância aos exemplos mal classificados pelos anteriores.

3. Stacking

No stacking, diferentes tipos de modelos (não necessariamente do mesmo tipo) são treinados. Em vez de simplesmente combinar suas previsões, um outro modelo (chamado de meta-modelo) é treinado para aprender a melhor forma de combinar as previsões desses modelos base.

Primeiro, vamos encontrar os melhores hiperparâmetros de cada modelo que vamos usar no Ensemble Model

In [ ]:
xgb_model = xgb.XGBClassifier(eval_metric='mlogloss', tree_method='hist', device='cuda')

param_grid = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(3, 15),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'gamma': uniform(0, 0.5),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1),
}

random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_grid, n_iter=100, cv=5, verbose=2, random_state=42, n_jobs=-1, scoring=make_scorer(accuracy_score))

random_search.fit(X_train_df_treating_data_scaled, y_train_df_treating_data)

best_params_xgb_model = random_search.best_params_
print("Best parameters found:", best_params_xgb_model)
print("Melhor score (RandomizedSearchCV):", random_search.best_score_)

Output do código acima:

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters found: {'colsample_bytree': np.float64(0.8769744131561081), 'gamma': np.float64(0.13470616689926074), 'learning_rate': np.float64(0.08323765667433225), 'max_depth': 13, 'n_estimators': 473, 'reg_alpha': np.float64(0.21876421957307024), 'reg_lambda': np.float64(0.5581020020173412), 'subsample': np.float64(0.7615344684232164)}

Melhor score (RandomizedSearchCV): 0.8152882205513784

In [ ]:
lgb_model = lgb.LGBMClassifier()

param_grid = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(3, 15),
    'num_leaves': randint(20, 200),
    'learning_rate': uniform(0.01, 0.3),
    'colsample_bytree': uniform(0.6, 0.4),
    'subsample': uniform(0.6, 0.4),
    'min_child_weight': uniform(0.01, 0.1),
    'min_split_gain': uniform(0, 0.1),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1),
}

random_search = RandomizedSearchCV(
    estimator=lgb_model,
    param_distributions=param_grid,
    n_iter=100,
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1,
    scoring=make_scorer(accuracy_score)
)

random_search.fit(X_train_df_treating_data_scaled, y_train_df_treating_data)

best_params_lgb_model = random_search.best_params_
print("Best parameters found:", best_params_lgb_model)
print("Best score (RandomizedSearchCV):", random_search.best_score_)

Output do código acima:

Best parameters found: {'colsample_bytree': np.float64(0.6943939678995823), 'learning_rate': np.float64(0.08682049682839718), 'max_depth': 13, 'min_child_weight': np.float64(0.06227328293819941), 'min_split_gain': np.float64(0.042754101835854964), 'n_estimators': 653, 'num_leaves': 143, 'reg_alpha': np.float64(0.10789142699330445), 'reg_lambda': np.float64(0.03142918568673425), 'subsample': np.float64(0.8545641645055122)}

Best score (RandomizedSearchCV): 0.811904761904762

In [ ]:
logistic_model = LogisticRegression(class_weight='balanced', random_state=42)

param_distributions = {
    'C': loguniform(1e-3, 1e3),
    'penalty': ['l1', 'l2', 'elasticnet'],
    'solver': ['saga'],
    'max_iter': [100, 500, 1000, 1500],
    'l1_ratio': uniform(0, 1)
}

scoring = { 'accuracy': make_scorer(accuracy_score), 'precision': make_scorer(precision_score, average='weighted'), 'recall': make_scorer(recall_score, average='weighted'), 'f1': make_scorer(f1_score, average='weighted')}

random_search = RandomizedSearchCV( estimator=logistic_model, param_distributions=param_distributions, n_iter=100, scoring=scoring, refit='f1', cv=5, verbose=2, n_jobs=-1,
random_state=42 )

random_search.fit(X_train_df_treating_data_scaled, y_train_df_treating_data)

print("Best parameters found:", random_search.best_params_)
print("Best F1 score:", random_search.best_score_)

Output do código acima:

Best parameters found: {'C': np.float64(93.84800715909533), 'l1_ratio': np.float64(0.3567533266935893), 'max_iter': 1500, 'penalty': 'l1', 'solver': 'saga'}

Best F1 score: 0.7550209677454942

In [ ]:
knn = KNeighborsClassifier()

param_distributions = {
    'n_neighbors': randint(1, 30),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2],
    'leaf_size': randint(10, 50),
    'metric': ['minkowski', 'euclidean', 'manhattan']
}

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1': make_scorer(f1_score, average='weighted')
}

random_search = RandomizedSearchCV(
    estimator=knn,
    param_distributions=param_distributions,
    n_iter=100,
    scoring=scoring,
    refit='f1',
    cv=5,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train_df_treating_data_scaled, y_train_df_treating_data)

print("Best parameters found:", random_search.best_params_)
print("Best F1 score:", random_search.best_score_)

results = random_search.cv_results_
for metric in scoring.keys():
    print(f"Best {metric} score: {results[f'mean_test_{metric}'][random_search.best_index_]:.4f}")

Output do código acima:

Best parameters found: {'algorithm': 'auto', 'leaf_size': 24, 'metric': 'manhattan', 'n_neighbors': 26, 'p': 2, 'weights': 'distance'}

Best F1 score: 0.7838104588532426

Best accuracy score: 0.7839

Best precision score: 0.7840

Best recall score: 0.7839

Best f1 score: 0.7838


In [ ]:
dt_model = DecisionTreeClassifier(random_state=42)

param_distributions = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': randint(3, 20),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': [None, 'sqrt', 'log2'] + list(uniform(0.1, 0.9).rvs(10)),
    'class_weight': [None, 'balanced'],
    'ccp_alpha': uniform(0, 0.05)
}

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1': make_scorer(f1_score, average='weighted')
}

random_search = RandomizedSearchCV(
    estimator=dt_model,
    param_distributions=param_distributions,
    n_iter=100,
    scoring=scoring,
    refit='f1',
    cv=5,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train_df_treating_data_scaled, y_train_df_treating_data)

print("Best parameters found:", random_search.best_params_)
print("Best F1 score:", random_search.best_score_)

results = random_search.cv_results_
for metric in scoring.keys():
    print(f"Best {metric} score: {results[f'mean_test_{metric}'][random_search.best_index_]:.4f}")

Output do código acima:

Best parameters found: {'ccp_alpha': np.float64(0.015239062907901453), 'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 11, 'max_features': np.float64(0.6087974157673947), 'min_samples_leaf': 1, 'min_samples_split': 11, 'splitter': 'random'}

Best F1 score: 0.7465959855465126

Best accuracy score: 0.7472

Best precision score: 0.7512

Best recall score: 0.7472

Best f1 score: 0.7466

In [48]:
xgb_model = xgb.XGBClassifier(colsample_bytree=0.8769744131561081,gamma=0.13470616689926074,learning_rate=0.08323765667433225,max_depth=13,n_estimators=473,reg_alpha=0.21876421957307024,reg_lambda=0.5581020020173412,subsample=0.7615344684232164,use_label_encoder=False,eval_metric='mlogloss')

lgb_model = lgb.LGBMClassifier(colsample_bytree=0.6943939678995823,learning_rate=0.08682049682839718,max_depth=13,min_child_weight=0.06227328293819941,min_split_gain=0.042754101835854964,n_estimators=653,num_leaves=143,reg_alpha=0.10789142699330445,reg_lambda=0.03142918568673425,subsample=0.8545641645055122)

logistic_model = LogisticRegression(C=93.84800715909533,l1_ratio=0.3567533266935893,max_iter=1500,penalty='l1',solver='saga')

knn_model = KNeighborsClassifier(algorithm='auto',leaf_size=24,metric='manhattan',n_neighbors=26,p=2,weights='distance')

dt_model = DecisionTreeClassifier(ccp_alpha=0.015239062907901453,class_weight='balanced',criterion='entropy',max_depth=11,max_features=0.6087974157673947,min_samples_leaf=1, min_samples_split=11,splitter='random')

Agora, vamos criar nosso Ensemble Model:

In [ ]:
ensemble_model = VotingClassifier(
    estimators=[
        ('rf_classifier', rf_classifier),
        ('xgb_model', xgb_model),
        ('lgb_model', lgb_model),
        ('logistic_model', logistic_model),
        ('knn_model', knn_model),
        ('dt_model', dt_model)
    ],
    voting='soft'
)

ensemble_model.fit(X_train_df_treating_data_scaled, y_train_df_treating_data)

In [ ]:
cv_scores = cross_val_score(ensemble_model, X_train_df_treating_data_scaled, y_train_df_treating_data, cv=5, scoring='accuracy')
print(f"Cross-validation accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

In [ ]:
y_test = ensemble_model.predict(X_val_df_treating_data_scaled)

print(classification_report(y_val_df_treating_data, y_test))

In [ ]:
predict_essemple = ensemble_model.predict(df_test_treated_scaled)
final_result_ensemble = pd.DataFrame({ 'track_unique_id': df_test['track_unique_id'], 'popularity_target': predict_essemple })
final_result_ensemble.to_csv('predictions_ensemble.csv', index=False)

Como podemos ver, o resulta é um pouco melhor do que com o modelo sozinho.

In [ ]:
rf_classifier.feature_importances_

Como podemos ver acima, a última feature está tomando toda a importância dos nossos dados e por isso não conseguimos progredir para além de 82%. Acredito que por causa do target_encode que fizemos no começo do nosso trabalho. Por isso, como um teste, vamos testar uma última forma para prever os resultados e ver como funciona.

### Usando Target Encode em toda base de dados

blablabla

In [159]:
def normalize_and_encode_dataset(df, target, n_splits=5):
    encoded_df = df.copy()
    encoding_dict = {}
    scaler_dict = {}

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    for column in df.columns:
        if column == target:
            continue
        
        encoded = np.zeros(len(df))
        column_dict = {}
        
        # Target encoding
        for train_idx, val_idx in kf.split(df):
            target_means = df.iloc[train_idx].groupby(column)[target].mean()
            encoded[val_idx] = df[column].iloc[val_idx].map(target_means)
            
            column_dict.update(target_means.to_dict())
        
        global_mean = df[target].mean()
        encoded = np.where(np.isnan(encoded), global_mean, encoded)
        
        column_dict['_global_mean'] = global_mean
        encoding_dict[column] = column_dict
        
        # Normalization
        scaler = StandardScaler()
        normalized = scaler.fit_transform(encoded.reshape(-1, 1)).flatten()
        
        encoded_df[f"{column}_encoded_normalized"] = normalized
        scaler_dict[column] = scaler
    
    return encoded_df, encoding_dict, scaler_dict

In [181]:
def apply_normalize_and_encode(df, encoding_dict, scaler_dict):
    encoded_df = df.copy()
    
    for column, column_dict in encoding_dict.items():
        if column in df.columns:
            global_mean = column_dict['_global_mean']
            
            # Apply target encoding
            encoded = df[column].map(column_dict)
            encoded = encoded.fillna(global_mean)
            
            # Apply normalization
            scaler = scaler_dict[column]
            normalized = scaler.transform(encoded.values.reshape(-1, 1)).flatten()
            
            encoded_df[f"{column}_encoded_normalized"] = normalized
        else:
            print(f"Warning: Column '{column}' not found in the input DataFrame.")
    
    return encoded_df

In [160]:
new_df = df.copy()
new_df = new_df.drop(columns=['track_unique_id', 'track_id'])

In [161]:
encoded_df, encoding_dict, scaler_dict = normalize_and_encode_dataset(new_df, 'popularity_target')

In [163]:
encoded_df = encoded_df.drop(columns=['artists', 'album_name', 'track_name', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre'])

In [165]:
X = encoded_df.drop('popularity_target', axis=1)
y = encoded_df['popularity_target']

In [ ]:
X_temp_df_no_genres, X_test_df_no_genres, y_temp_df_no_genres, y_test_df_no_genres = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_df_no_genres, X_val_df_no_genres, y_train_df_no_genres, y_val_df_no_genres = train_test_split(X_temp_df_no_genres, y_temp_df_no_genres, test_size=0.25, random_state=42)

print(f"Tamanho do conjunto de treino: {len(X_train_df_no_genres)}")
print(f"Tamanho do conjunto de validação: {len(X_val_df_no_genres)}")
print(f"Tamanho do conjunto de teste: {len(X_test_df_no_genres)}")

In [191]:
rf_classifier.fit(X_train_df_no_genres, y_train_df_no_genres)

y_pred_df_no_genres = rf_classifier.predict(X_val_df_no_genres)

print(classification_report(y_val_df_no_genres, y_pred_df_no_genres))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91      8178
           1       0.90      0.92      0.91      7782

    accuracy                           0.91     15960
   macro avg       0.91      0.91      0.91     15960
weighted avg       0.91      0.91      0.91     15960



In [ ]:
y_pred_df_no_genres = rf_classifier.predict(X_test_df_no_genres)

print(classification_report(y_test_df_no_genres, y_pred_df_no_genres))

In [ ]:
xgb_model.fit(X_train_df_no_genres, y_train_df_no_genres)

y_pred_df_no_genres = xgb_model.predict(X_val_df_no_genres)

print(classification_report(y_val_df_no_genres, y_pred_df_no_genres))

In [ ]:
y_pred_df_no_genres = xgb_model.predict(X_test_df_no_genres)

print(classification_report(y_test_df_no_genres, y_pred_df_no_genres))

In [ ]:
lgb_model.fit(X_train_df_no_genres, y_train_df_no_genres)

y_pred_df_no_genres = lgb_model.predict(X_val_df_no_genres)

print(classification_report(y_val_df_no_genres, y_pred_df_no_genres))

In [ ]:
y_pred_df_no_genres = lgb_model.predict(X_test_df_no_genres)

print(classification_report(y_test_df_no_genres, y_pred_df_no_genres))

In [190]:
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'max_features': ['auto', 'sqrt', 'log2'],
}


n_iter_search = 100  
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=n_iter_search, cv=5, scoring=make_scorer(accuracy_score), n_jobs=-1, random_state=42, verbose=2)

random_search.fit(X_train_df_no_genres, y_train_df_no_genres)

print("Melhores parâmetros (RandomizedSearchCV):", random_search.best_params_)
print("Melhor score (RandomizedSearchCV):", random_search.best_score_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


c:\Inteli\Mod03\PonderadaSemana04\proximo-hit-spotify\Ponderada\.venv\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
135 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
115 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Inteli\Mod03\PonderadaSemana04\proximo-hit-spotify\Ponderada\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Inteli\Mod03\PonderadaSemana04\proximo-hit-spotify\Ponderada\.venv\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Inteli\Mod03\PonderadaSemana04\proxim

Melhores parâmetros (RandomizedSearchCV): {'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None}
Melhor score (RandomizedSearchCV): 0.9055346700083543


Melhores parâmetros (RandomizedSearchCV): {'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None}
Melhor score (RandomizedSearchCV): 0.9055346700083543

In [184]:
df_test_target_encoding = df_test.copy()
df_test_target_encoding = df_test_target_encoding.drop(columns=['track_unique_id', 'track_id'])

In [185]:
df_test_target_encoding = apply_normalize_and_encode(df_test_target_encoding, encoding_dict, scaler_dict)

In [187]:
df_test_target_encoding = df_test_target_encoding.drop(columns=['artists', 'album_name', 'track_name', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre'])

In [189]:
predict_target_encoding = rf_classifier.predict(df_test_target_encoding)
result = pd.DataFrame({ 'track_unique_id': df_test['track_unique_id'], 'popularity_target': predict_target_encoding })
result.to_csv('result.csv', index=False)

---

Melhores parâmetros (RandomizedSearchCV): {'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None}
Melhor score (RandomizedSearchCV): 0.9055346700083543